In [2]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from selenium.common.exceptions import NoSuchElementException
from concurrent.futures import ThreadPoolExecutor
from time import sleep
import os
from selenium import webdriver
from selenium.webdriver.common.by import By
import time
import random
import pandas as pd
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
import json


In [2]:

# Đường dẫn lưu bookmark
BOOKMARK_PATH = r"D:/du lieu o cu/HUTECH Courses/NCKH(thầy Hường)/E-Commerce-Data-Analysis/Data Scraping Selenium/bookmark.csv"

# Thư mục lưu file kết quả
OUTPUT_DIR = r"D:/du lieu o cu/HUTECH Courses/NCKH(thầy Hường)/E-Commerce-Data-Analysis/Data Scraping Selenium/Category_Files"
if not os.path.exists(OUTPUT_DIR):
    os.makedirs(OUTPUT_DIR)

def load_bookmark(file_path):
    """Load danh sách liên kết đã cào từ file."""
    if os.path.exists(file_path):
        return set(pd.read_csv(file_path)['Links'])
    return set()

def save_bookmark(file_path, links):
    """Lưu danh sách liên kết đã cào vào file (thêm mới, không ghi đè)."""
    if os.path.exists(file_path):
        existing_links = load_bookmark(file_path)
        links = links.union(existing_links)  # Gộp liên kết cũ và mới
    df = pd.DataFrame({'Links': list(links)})
    df.to_csv(file_path, index=False)

def save_all_store_links(store_links):
    """Lưu tất cả các đường link cửa hàng vào một file Excel chung."""
    output_file = os.path.join(OUTPUT_DIR, "store_links.xlsx")
    df = pd.DataFrame({'Links': list(store_links)})
    df.to_excel(output_file, index=False)
    print(f"Đã lưu tất cả các liên kết cửa hàng vào file: {output_file}")

def scroll_page(driver, scroll_count=10, pause=1.5, offset=25):
    """Cuộn trang để tải thêm nội dung."""
    total_height = driver.execute_script("return document.body.scrollHeight")
    scroll_step = (total_height - offset) / scroll_count
    for i in range(1, scroll_count + 1):
        driver.execute_script(f"window.scrollTo(0, {i * scroll_step});")
        time.sleep(pause)
    time.sleep(2)  # Đợi 2 giây sau khi cuộn xong

def collect_product_links(driver, product_links):
    """Thu thập liên kết sản phẩm từ trang hiện tại."""
    product_elements = driver.find_elements(By.CSS_SELECTOR, ".styles__ProductItemContainerStyled-sc-bszvl7-0 a.style__ProductLink-sc-139nb47-2")
    for elem in product_elements:
        link = elem.get_attribute("href")
        if link and link not in product_links:
            product_links.add(link)
            print(f"Collected {len(product_links)} links")

def crawl_category_links(driver, category_links):
    """
    Duyệt qua các danh mục và thu thập các liên kết cửa hàng.
    Thu thập liên tục cho đến khi không còn nút "Xem thêm" hoặc không có sản phẩm mới.
    """
    all_store_links = set()

    for idx, href in enumerate(category_links, start=1):
        print(f"\nĐang truy cập danh mục {idx}: {href}")
        driver.get(href)
        time.sleep(random.uniform(2, 5))

        category_product_links = set()

        while True:
            # Lưu số lượng hiện tại để so sánh sau khi thu thập
            current_count = len(category_product_links)
            collect_product_links(driver, category_product_links)
            
            # Nếu không có liên kết mới được thêm, có thể đã tải hết sản phẩm hiện tại
            if len(category_product_links) == current_count:
                try:
                    # Thử tìm và click nút "Xem thêm" nếu có
                    button = driver.find_element(By.CSS_SELECTOR, '[data-view-id="category_infinity_view.more"]')
                    ActionChains(driver).move_to_element(button).click(button).perform()
                    time.sleep(random.uniform(2, 3))
                    scroll_page(driver, scroll_count=5, pause=random.uniform(1, 1.5))
                except NoSuchElementException:
                    print("Nút 'Xem thêm' không còn hoặc đã tải hết sản phẩm.")
                    break
            else:
                # Nếu có sản phẩm mới được thêm, thử cuộn trang để kích hoạt load thêm
                scroll_page(driver, scroll_count=5, pause=random.uniform(1, 1.5))
        
        print(f"Danh mục {idx} hoàn thành. Thu thập được {len(category_product_links)} liên kết.")
        # Cập nhật tất cả liên kết cửa hàng với liên kết từ danh mục hiện tại
        all_store_links = all_store_links.union(category_product_links)

    return all_store_links

def main():
    # Tải bookmark đã cào (nếu có)
    crawled_links = load_bookmark(BOOKMARK_PATH)

    # Khởi tạo WebDriver
    driver = webdriver.Chrome()
    driver.get("https://tiki.vn")
    driver.implicitly_wait(5)

    # Tìm tất cả các thẻ <a> bên trong class "styles__StyledListItem-sc-w7gnxl-0"
    category_elements = driver.find_elements(By.CSS_SELECTOR, ".styles__StyledListItem-sc-w7gnxl-0 a")
    category_links = [link.get_attribute("href") for link in category_elements if link.get_attribute("href") not in crawled_links]
    category_links = category_links[1:26]  # Lấy danh mục từ 2 đến 26

    # Thu thập liên kết sản phẩm từ các danh mục và gom chúng lại
    all_store_links = crawl_category_links(driver, category_links)

    print("\nHoàn thành thu thập liên kết sản phẩm.")
    print(f"Tổng số liên kết thu thập được: {len(all_store_links)}")

    # Lưu tất cả các liên kết cửa hàng vào 1 file Excel chung
    save_all_store_links(all_store_links)

    # Nếu cần cập nhật bookmark thì lưu lại các liên kết danh mục đã cào (tuỳ chọn)
    save_bookmark(BOOKMARK_PATH, set(category_links))

    # Đóng trình duyệt
    driver.quit()

if __name__ == "__main__":
    main()



Đang truy cập danh mục 1: https://tiki.vn/nha-cua-doi-song/c1883
Collected 1 links
Collected 2 links
Collected 3 links
Collected 4 links
Collected 5 links
Collected 6 links
Collected 7 links
Collected 8 links
Collected 9 links
Collected 10 links
Collected 11 links
Collected 12 links
Collected 13 links
Collected 14 links
Collected 15 links
Collected 16 links
Collected 17 links
Collected 18 links
Collected 19 links
Collected 20 links
Collected 21 links
Collected 22 links
Collected 23 links
Collected 24 links
Collected 25 links
Collected 26 links
Collected 27 links
Collected 28 links
Collected 29 links
Collected 30 links
Collected 31 links
Collected 32 links
Collected 33 links
Collected 34 links
Collected 35 links
Collected 36 links
Collected 37 links
Collected 38 links
Collected 39 links
Collected 40 links
Collected 41 links
Collected 42 links
Collected 43 links
Collected 44 links
Collected 45 links
Collected 46 links
Collected 47 links
Collected 48 links
Collected 49 links
Collected 50

Truy cập để lấy LINK CỬA HÀNG

In [5]:
import pandas as pd
import threading
from concurrent.futures import ThreadPoolExecutor
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException, TimeoutException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from openpyxl import load_workbook

# Đường dẫn file Excel
input_file = r"D:/du lieu o cu/HUTECH Courses/NCKH(thầy Hường)/E-Commerce-Data-Analysis/Data Scraping Selenium/Category_Files/store_links.xlsx"
output_file = r"D:/du lieu o cu/HUTECH Courses/NCKH(thầy Hường)/E-Commerce-Data-Analysis/Data Scraping Selenium/Category_Files/store_seller_links_1.xlsx"
error_file = r"D:/du lieu o cu/HUTECH Courses/NCKH(thầy Hường)/E-Commerce-Data-Analysis/Data Scraping Selenium/Category_Files/Still not collected seller yet_1.xlsx"

# Đọc file Excel và lấy 20 dòng đầu tiên
df = pd.read_excel(input_file)
links = df["Links"].tolist()[13700:14000]
# 1800:2500
# Sử dụng tập hợp để tránh trùng lặp seller links
seller_links_set = set()

# Danh sách lưu các URL bị lỗi
error_links = []

# Khóa để đảm bảo ghi file an toàn khi dùng đa luồng
lock = threading.Lock()

# Hàm xử lý từng link
def process_link(url):
    global seller_links_set, error_links

    # Khởi tạo Selenium WebDriver trong từng luồng
    driver = webdriver.Chrome()
    driver.maximize_window()
    driver.implicitly_wait(10)

    try:
        print(f"🔹 Đang truy cập: {url}")
        driver.get(url)

        # **Thực hiện reload trang sau khi truy cập**
        driver.refresh()

        # Chờ trang load hoàn toàn sau khi reload (timeout 10s)
        WebDriverWait(driver, 10).until(
            # EC.presence_of_element_located((By.CSS_SELECTOR, "span.seller-name div.sc-d1e96cea-0.jBNFDc a"))
            EC.presence_of_element_located((By.CSS_SELECTOR, "span.seller-name div.SellerName__SellerNameStyled-sc-5d1cxl-0.myqYs a"))

        )

        # Tìm seller link
        # seller_element = driver.find_element(By.CSS_SELECTOR, "span.seller-name div.sc-d1e96cea-0.jBNFDc a")
        seller_element = driver.find_element(By.CSS_SELECTOR, "span.seller-name div.SellerName__SellerNameStyled-sc-5d1cxl-0.myqYs a")

        seller_href = seller_element.get_attribute("href")
        print(f"  ✅ Seller link: {seller_href}")

        # Dùng khóa để đảm bảo an toàn khi ghi file
        with lock:
            if seller_href not in seller_links_set:
                seller_links_set.add(seller_href)

                # Đọc file hiện tại (nếu có)
                try:
                    existing_df = pd.read_excel(output_file)
                except FileNotFoundError:
                    existing_df = pd.DataFrame(columns=["Seller Links"])

                # Thêm dữ liệu mới
                new_df = pd.DataFrame({"Seller Links": [seller_href]})
                final_df = pd.concat([existing_df, new_df], ignore_index=True)

                # Ghi vào file Excel
                with pd.ExcelWriter(output_file, mode="w", engine="openpyxl") as writer:
                    final_df.to_excel(writer, index=False)

    except TimeoutException:
        print(f"  ⚠️ Trang {url} load quá lâu, bỏ qua.")
        save_error(url, "Load quá lâu")
    except NoSuchElementException:
        print(f"  ❌ Không tìm thấy seller link trên trang: {url}")
        save_error(url, "Không tìm thấy seller link")
    except Exception as e:
        print(f"  ❌ Lỗi khi xử lý {url}: {e}")
        save_error(url, str(e))
    finally:
        driver.quit()

# Hàm lưu các URL bị lỗi vào file "Still not collected seller yet.xlsx"
def save_error(url, error_message):
    with lock:
        try:
            existing_error_df = pd.read_excel(error_file)
        except FileNotFoundError:
            existing_error_df = pd.DataFrame(columns=["URL", "Lỗi"])

        new_error_df = pd.DataFrame({"URL": [url], "Lỗi": [error_message]})
        final_error_df = pd.concat([existing_error_df, new_error_df], ignore_index=True)

        with pd.ExcelWriter(error_file, mode="w", engine="openpyxl") as writer:
            final_error_df.to_excel(writer, index=False)

# Sử dụng ThreadPoolExecutor để chạy đa luồng
max_threads = 5  # Số luồng tối đa có thể điều chỉnh
with ThreadPoolExecutor(max_workers=max_threads) as executor:
    executor.map(process_link, links)

print(f"\n✅ Đã hoàn thành. Dữ liệu lỗi được lưu vào: {error_file}")


🔹 Đang truy cập: https://tiki.vn/giay-loc-ca-phe-hario-100-cai-v60-vcf-02-100w-p117059826.html?spid=117059827
🔹 Đang truy cập: https://tiki.vn/dien-thoai-vivo-y18-4gb-128gb-hang-chinh-hang-p275775887.html?spid=275775888
🔹 Đang truy cập: https://tiki.vn/tui-chong-soc-laptop-macbook-agva-jersey-13inch-kich-thuoc-35-x-2-5-x-26-cm-ma-san-pham-slv338-3-mau-xam-xanh-den-p153978796.html?spid=153978810
🔹 Đang truy cập: https://tiki.vn/ruou-blended-whisky-johnnie-walker-double-black-40-750ml-co-hop-p186267263.html?spid=186267265
🔹 Đang truy cập: https://tiki.vn/thiet-bi-phat-song-wifi-tenda-n300-hang-chinh-hang-p950026.html?spid=272455472
  ✅ Seller link: https://tiki.vn/cua-hang/the-gioi-di-dong-official?source_screen=product_detail&source_engine=organic
  ✅ Seller link: https://tiki.vn/cua-hang/hario-official-store?source_screen=product_detail&source_engine=organic
  ✅ Seller link: https://tiki.vn/cua-hang/vua-balo?source_screen=product_detail&source_engine=organic
  ✅ Seller link: https://ti

In [ ]:
import pandas as pd
df_new = pd.read_excel(r"D:/du lieu o cu/HUTECH Courses/NCKH(thầy Hường)/E-Commerce-Data-Analysis/Data Scraping Selenium/Category_Files/store_seller_links.xlsx")
df_new.drop_duplicates(subset ="Seller Links", keep = 'first', inplace = True)
df_new

,Seller Links
0,https://tiki.vn/cua-hang/cong-nghe-viet?source...
1,https://tiki.vn/cua-hang/lynu-store?source_scr...
2,https://tiki.vn/cua-hang/kho-si-cong-nghe?sour...
3,https://tiki.vn/cua-hang/telovietnam?source_sc...
4,https://tiki.vn/cua-hang/rem-che-nang-thiet-ke...
...,...
6546,https://tiki.vn/cua-hang/thao-nguyen-phat?sour...
6552,https://tiki.vn/cua-hang/den-bat-muoi-con-doi?...
6564,https://tiki.vn/cua-hang/cong-ty-hue-minh?sour...
6570,https://tiki.vn/cua-hang/nha-sach-phuong-nam?s...


In [ ]:
!pip install gspread oauth2client pandas


   ---------------------------------------- 0.0/57.6 kB ? eta -:--:--
   ---------------------------------------- 0.0/57.6 kB ? eta -:--:--
   --------------------- ------------------ 30.7/57.6 kB 1.3 MB/s eta 0:00:01
   ---------------------------------------- 57.6/57.6 kB 751.3 kB/s eta 0:00:00
   ---------------------------------------- 0.0/98.2 kB ? eta -:--:--
   ---------------------------------------- 98.2/98.2 kB 2.8 MB/s eta 0:00:00
   ---------------------------------------- 0.0/210.8 kB ? eta -:--:--
   --------------------------- ------------ 143.4/210.8 kB 4.3 MB/s eta 0:00:01
   ---------------------------------------- 210.8/210.8 kB 2.6 MB/s eta 0:00:00
   ---------------------------------------- 0.0/96.9 kB ? eta -:--:--
   ---------------------------------------- 96.9/96.9 kB 5.8 MB/s eta 0:00:00
   ---------------------------------------- 0.0/151.7 kB ? eta -:--:--
   ---------------------------------------- 151.7/151.7 kB 4.6 MB/s eta 0:00:00



[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Lấy Link của từng sản phẩm trong store

In [ ]:
import os
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException, WebDriverException, ElementClickInterceptedException


# Hàm cuộn trang có giới hạn (chừa lại 1/7 trang)
def scroll_page(driver, scroll_count=10, pause=1.5, offset=25):
    """Cuộn trang để tải thêm nội dung."""
    total_height = driver.execute_script("return document.body.scrollHeight")
    scroll_step = (total_height - offset) / scroll_count
    for i in range(1, scroll_count + 1):
        driver.execute_script(f"window.scrollTo(0, {i * scroll_step});")
        time.sleep(pause)
    time.sleep(2)  # Đợi 2 giây sau khi cuộn xong

# Hàm nhấn nút "Xem thêm" nếu còn
def click_view_more(driver):
    """Tìm và nhấn vào nút 'Xem Thêm' nếu có."""
    try:
        while True:
            view_more_button = driver.find_element(By.CSS_SELECTOR, "button.ViewMoreBtn__Wrapper-sc-qs9ydg-0")
            if view_more_button.is_displayed():
                driver.execute_script("arguments[0].click();", view_more_button)
                # print("Đã nhấn 'Xem Thêm'. Chờ tải thêm sản phẩm...")
                time.sleep(3)  # Đợi trang tải
            else:
                break
    except (NoSuchElementException, ElementClickInterceptedException):
        # print("Không tìm thấy hoặc không thể ấn nút 'Xem Thêm'.")
        return


# Đường dẫn file chứa danh sách cửa hàng
input_file = r"D:/du lieu o cu/HUTECH Courses/NCKH(thầy Hường)/E-Commerce-Data-Analysis/Data Scraping Selenium/Category_Files/store_seller_links.xlsx"

# Đọc danh sách seller links
df = pd.read_excel(input_file)
seller_links = df["Seller Links"].dropna().tolist()[:1]

# Khởi tạo trình duyệt
driver = webdriver.Chrome()
driver.maximize_window()
driver.implicitly_wait(10)  # Chờ tối đa 10 giây

# Danh sách lưu thông tin
data = []

# Duyệt từng cửa hàng
for link in seller_links:
    try:
        print(f"Đang truy cập: {link}")
        try:
            driver.get(link)
            time.sleep(3)  # Chờ trang tải
        except WebDriverException as e:
            print(f"Lỗi khi mở trang {link}: {e}")
            continue

        # Bước 1: Lấy tên cửa hàng
        try:
            # store_name_element = driver.find_element(By.CSS_SELECTOR, "h1.sc-b4c2fa32-2.fXWGFK")
            store_name_element = driver.find_element(By.CSS_SELECTOR, "h1.Header__SellerName-sc-vgcg69-2.kgqRrB")
            store_name = store_name_element.text.strip()
            print(f"Tên cửa hàng: {store_name}")
        except NoSuchElementException:
            store_name = None
            print("Không tìm thấy tên cửa hàng.")


        # Bước 2: Click vào tab "Hồ Sơ Cửa Hàng"
        try:
            profile_tab = driver.find_element(By.XPATH, "//div[contains(text(), 'Hồ Sơ Cửa Hàng')]")
            profile_tab.click()
            time.sleep(3)  # Chờ trang tải lại sau khi click
            print("Đã click vào tab 'Hồ Sơ Cửa Hàng'")
        except NoSuchElementException:
            print("Không tìm thấy tab 'Hồ Sơ Cửa Hàng', bỏ qua trang này.")
            continue

        # Bước 3: Lấy thông tin cửa hàng
        try:
            # info_values = driver.find_elements(By.CSS_SELECTOR, 
            #     "span.sc-ba05c94e-5.igQJxN, "
            #     "span.sc-ba05c94e-4.fAQpWx")
            info_values = driver.find_elements(By.CSS_SELECTOR, 
                "span.StoreInfo__InfoValue-sc-1un24du-5.GxEKI, "
                "span.StoreInfo__InfoLabel-sc-1un24du-4.jpeWpY")

            extracted_values = [value.text.strip() for value in info_values if value.text.strip()]
            # print(f"Thông tin cửa hàng: {extracted_values}")

            year = extracted_values[2]
            total_products = extracted_values[5] 
            rating = extracted_values[11][:3]
            review_count = extracted_values[12]
            followers = extracted_values[15] 
            chat_response = extracted_values[18]
            print(year, total_products, rating, review_count, followers, chat_response)
            
        except Exception as e:
            print(f"Lỗi khi lấy thông tin cửa hàng {link}: {e}")
            year, total_products, rating, review_count, followers, chat_response = (None, None, None, None, None, None)


        # Bước 4: Click vào tab "Tất Cả Sản Phẩm"
        try:
            all_products_tab = driver.find_element(By.XPATH, "//div[contains(text(), 'Tất Cả Sản Phẩm')]")
            all_products_tab.click()
            time.sleep(3)  # Chờ trang tải lại sau khi click
            # print("Đã click vào tab 'Tất Cả Sản Phẩm'")
        except NoSuchElementException:
            print("Không tìm thấy tab 'Tất Cả Sản Phẩm', bỏ qua sản phẩm.")
            continue


        # Bước 5: Cuộn trang và nhấn "Xem thêm"
        scroll_page(driver)
        # print("Đã cuộn trang. Đang lấy thông tin sản phẩm...")
        click_view_more(driver)
        # print("Đã click viewmore.")

        # Bước 4: Lấy thông tin từng sản phẩm (Tên, Giá, Số lượng bán)
        try:
            # product_elements = driver.find_elements(By.CSS_SELECTOR, "a.sc-e14980c3-2.sc-e14980c3-3.hQMtFc")
            product_elements = driver.find_elements(By.CSS_SELECTOR, "a.Product__Wrapper-sc-n99tp2-2.Product___StyledWrapper-sc-n99tp2-3.fNzCgC")

            for product in product_elements:
                try:
                    # Lấy tên sản phẩm (nằm trong thẻ <h3> bên trong div chứa class "kuTTXp")
                    # product_name = product.find_element(By.CSS_SELECTOR, "div.sc-e14980c3-7.kuTTXp > h3").text.strip()
                    product_name = product.find_element(By.CSS_SELECTOR, "div.Product__Name-sc-n99tp2-7.cfPcgv > h3").text.strip()
                    print(product_name)
                except Exception as e:
                    product_name = None
                    print("Không lấy được tên sản phẩm:", e)

                # Lấy href (đường dẫn chi tiết sản phẩm từ thuộc tính href của thẻ <a>)
                product_link = product.get_attribute("href")
                print(product_link)
                
                try:
                    # Lấy rating: giá trị rating nằm trong div có class "sc-e14980c3-25 iUUFFW"
                    # product_rating = product.find_element(By.CSS_SELECTOR, "div.sc-e14980c3-25.iUUFFW").text.strip()
                    product_rating = product.find_element(By.CSS_SELECTOR, "div.Product__TextRating-sc-n99tp2-25.dDxtAi").text.strip()
                    print(product_rating)
                except Exception as e:
                    product_rating = None
                    print("Không lấy được rating:", e)

                try:
                    # Lấy số lượng đã bán: nằm trong thẻ <span class="percent">
                    sold_text = product.find_element(By.CSS_SELECTOR, "span.percent").text.strip()
                    # Loại bỏ chữ "Đã bán " nếu có
                    product_quantity_sold = sold_text.replace("Đã bán ", "")
                    print(product_quantity_sold)
                except Exception as e:
                    product_quantity_sold = None
                    print("Không lấy được số lượng đã bán:", e)
                
                try:
                    # Lấy giá sản phẩm: giá nằm trong thẻ <span> bên trong div có class "cspIwy has-discount"
                    product_price = product.find_element(By.CSS_SELECTOR, "div.Product__PriceDiscountValue-sc-n99tp2-9.eWvHOp span").text.strip()
                    print(product_price)
                except Exception as e:                                         
                    product_price = None
                    print("Không lấy được giá sản phẩm:", e)




                # Lưu dữ liệu vào danh sách
                data.append({
                    "Seller Link": link,
                    "Store Name": store_name,
                    "Year Joined": year,
                    "Total Products": total_products,
                    "Rating": rating,
                    "Review Count": review_count,
                    "Followers": followers,
                    "Chat Response Rate": chat_response,
                    "Product Name": product_name,
                    "Product Link": product_link,
                    "Product Rating": product_rating,
                    "Price (VND)": product_price,
                    "Sold Quantity": product_quantity_sold
                })

            print(data)
        except Exception as e:
            print(f"Lỗi khi lấy danh sách sản phẩm tại {link}: {e}")
            continue

    except Exception as e:
        print(f"Lỗi khi xử lý {link}: {e}")

# Đóng trình duyệt
driver.quit()

# Lưu vào Excel
output_file = r"D:/du lieu o cu/HUTECH Courses/NCKH(thầy Hường)/E-Commerce-Data-Analysis/Data Scraping Selenium/store_products.xlsx"
output_df = pd.DataFrame(data)
output_df.to_excel(output_file, index=False)

print(f"\nĐã lưu thông tin vào file: {output_file}")

In [ ]:
import pandas as pd

# Thiết lập để hiển thị tất cả các cột
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)  # Đảm bảo chiều rộng hiển thị không bị giới hạn
pd.set_option('display.max_colwidth', None)  # Hiển thị đầy đủ nội dung trong mỗi cột

df_new = pd.DataFrame(data)
display(df_new)


Truy cập vào từng sản phẩm trong store